# Implementação MLP para dataset

In [6]:
# Função de ativação Logit

def f_net(activation):
     return 1.0 / (1.0 + np.exp(-activation))

In [7]:
# Derivada da Logit

def f_prime_net(f_net):
    return f_net*(1-f_net)

In [8]:
# Etapa Foward

def f_Forward(Xp,w_hidden_layer,w_output_layer,net_hidden,fnet_hidden,net_out, fnet_out,n_hidden):
    # Camada oculta
    net_hidden  = np.sum(w_hidden_layer*Xp,axis=1)
    fnet_hidden = f_net(net_hidden)
    f_prime_hp = f_prime_net(fnet_hidden)
    # Camada de saída
    net_out = np.sum(np.r_[fnet_hidden, 1]*w_output_layer,axis=1)
    fnet_out =  f_net(net_out)
    f_prime_op = f_prime_net(fnet_out)
    
    return net_hidden, fnet_hidden, net_out, fnet_out, f_prime_op, f_prime_hp



In [20]:
# Backpropagation

def mlp_backpropagation(X,Y,n_inputs,n_hidden,n_outputs ,eta ,epoch,alpha):
    # Vetor de entrada com acréscimo de coluna para bias
    Xp = np.concatenate((X, np.reshape(np.ones(len(X)),(len(X),1))), axis=1)

    # Inicialização dos pesos de forma aleatória
    var_w_hidden = w_hidden_layer_ant =  w_hidden_layer = np.array([[random.uniform(-0.5,0.5) for i in range(n_inputs+1)] for i in range(n_hidden)])
    var_w_output = w_output_layer_ant =  w_output_layer = np.array([[random.uniform(-0.5,0.5) for i in range(n_hidden+1)]     for i in range(n_outputs)])

    ## Hidden Layer
    net_hidden  = np.zeros(w_hidden_layer.shape)
    fnet_hidden = np.zeros(w_hidden_layer.shape)

    ## Output Layer
    net_out  = np.zeros(n_outputs)
    fnet_out = np.zeros(n_outputs)
    # Vetor de saída da hidden com acréscimo de coluna para bias

    counter = 0
    
    while counter <= epoch :
        squaredError = 0
        for i in range(len(X)):
            net_hidden, fnet_hidden, net_out, fnet_out, f_prime_op, f_prime_hp = f_Forward(Xp[i],w_hidden_layer,w_output_layer,net_hidden,fnet_hidden,net_out, fnet_out,n_hidden)

          
            error = Y[i] - fnet_out
            squaredError = squaredError + np.sum(error**2)

            #Cálculo dos deltas
            delta_op  = error * f_prime_op        
            w_o_kj    = w_output_layer[:,0:w_output_layer.shape[1]-1]
            delta_h_p = f_prime_hp*np.dot(w_o_kj.T,delta_op)

            #Treinamento
            w_output_layer = w_output_layer + eta * np.dot(  np.reshape(delta_op,(len(w_output_layer),1)), np.reshape(np.r_[fnet_hidden,1],(1,n_hidden+1))) + alpha * var_w_output
            w_hidden_layer = w_hidden_layer + eta * np.dot(np.reshape(delta_h_p,(n_hidden,1)),np.reshape(Xp[i],(1,len(X[0])+1))) + alpha * var_w_hidden
            
            # Calculo do momento
            var_w_output = w_output_layer - w_output_layer_ant
            w_output_layer_ant = w_output_layer.copy()
            
            var_w_hidden = w_hidden_layer - w_hidden_layer_ant
            w_hidden_layer_ant = w_hidden_layer.copy()
      
            
        squaredError = squaredError / len(X)
        print(squaredError)
        counter = counter + 1
    
    return  squaredError , w_output_layer, w_hidden_layer, fnet_out

In [10]:
# Predição de novos dados

def f_predict(Xp,w_hidden_layer,w_output_layer):

    return  np.round(f_net(np.sum(np.r_[f_net(np.sum(w_hidden_layer*Xp.T,axis=1)), 1]*w_output_layer,axis=1)))

     
    


In [21]:
import numpy as np
from math import exp
import random
import pandas as pd
#tratamentos de dataframes
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

random.seed(123)

# Importação do conjunto de dados Wine sem cabeçalho
df = pd.read_csv("/home/jovyan/MLPxRBF/wine.csv",delimiter=',',header=None)

# Nomeação das colunas
dict_colnames = {
0: 'type',
1: 'alcohol',
2: 'malic_acid',
3: 'ash',
4: 'alcalinity_ash',
5: 'magnesium',
6: 'total_phenols',
7: 'flavanoids',
8: 'nonflavanoid_phenols',
9: 'proanthocyanins',
10: 'color_intensity',
11: 'hue',
12: 'OD280_OD315_diluted_wines',
13: 'proline'
   }
    
df = df.rename(columns= dict_colnames)

# Criação do vetor de base canônica
df['p1'] = np.where(df['type']==1, 1, 0)
df['p2'] = np.where(df['type']==2, 1, 0)
df['p3'] = np.where(df['type']==3, 1, 0)

# Separação entre dados de treino e teste
lista = ['type','p1','p2','p3']
lista2 = ['p1','p2','p3']

x_train, x_test, y_train, y_test = train_test_split(df.drop(lista, axis = 1), 
                                                    df[lista2], 
                                                    test_size = 0.3, 
                                                    random_state = random.seed(123))

# Normalização dos dados
x_scaled = preprocessing.scale(x_train)
scaler = preprocessing.StandardScaler().fit(x_train)
x_train_norm = scaler.transform(x_train)
x_test_norm = scaler.transform(x_test)


# Matriz para treinar a rede
X = x_train_norm.copy()
Y = np.array(y_train.copy())

## Dimensões da Rede
n_inputs = len(X[0])
n_outputs = len(Y[0])
n_hidden = 3


Erro , w_output_layer, w_hidden_layer, fnet_out = \
    mlp_backpropagation(X,Y,n_inputs,n_hidden,n_outputs ,eta = 0.01,epoch=100,alpha = 0.3)


0.7964423260004194
0.7473951342429703
0.7099591336483712
0.6815847476403972
0.6596053404783473
0.6419855131669091
0.6272870459733252
0.6145336328442627
0.6030766758829804
0.5924912722659138
0.5825028444497455
0.5729372784561518
0.5636873675357295
0.5546901355501891
0.5459113632306475
0.5373349326227836
0.5289554615619634
0.5207732389239405
0.5127908025124668
0.5050107047059494
0.49743413875397363
0.4900601829554986
0.4828854793885211
0.47590420849185405
0.4691082558187682
0.4624874952595354
0.4560301352729431
0.4497230919625619
0.4435523658018454
0.4375034080408789
0.43156146890518293
0.4257119232108446
0.4199405705541612
0.414233907372649
0.4085793675081533
0.40296552701878346
0.3973822684324214
0.39182089985367974
0.38627422558403457
0.3807365671875071
0.37520373694525117
0.36967296887490586
0.36414281532676945
0.3586130190599348
0.35308437132060655
0.3475585657654184
0.3420380563488642
0.3365259249590876
0.331025762116351
0.32554156184361543
0.3200776301339425
0.31463850537172605
0.

In [3]:
    # Inicialização dos pesos de forma aleatória
w_hidden_layer_ant =  w_hidden_layer = np.array([[random.uniform(-0.5,0.5) for i in range(n_inputs+1)] for i in range(n_hidden)])
w_output_layer_ant =  w_output_layer = np.array([[random.uniform(-0.5,0.5) for i in range(n_hidden+1)]     for i in range(n_outputs)])

In [16]:
w_hidden_layer_ant

array([[-0.16308385, -0.10880669, -0.42670922, -0.04323293,  0.07316851,
        -0.1638939 , -0.31093813, -0.41184271,  0.12831752, -0.48717089,
        -0.14808632, -0.27087024,  0.03199749,  0.22366335],
       [ 0.40915328,  0.13880084,  0.4917931 ,  0.46405917,  0.12999071,
        -0.26662682, -0.01618852, -0.32241027,  0.49195777, -0.05550008,
         0.25165016, -0.45587014, -0.3652724 , -0.30682303],
       [-0.24363765, -0.14795247, -0.22807015, -0.06573142, -0.07964268,
        -0.39137917, -0.43739123, -0.14493855, -0.4112111 , -0.06679969,
        -0.49143879,  0.03580102, -0.42651601,  0.19420591]])

In [5]:
w_hidden_layer

array([[-0.16308385, -0.10880669, -0.42670922, -0.04323293,  0.07316851,
        -0.1638939 , -0.31093813, -0.41184271,  0.12831752, -0.48717089,
        -0.14808632, -0.27087024,  0.03199749,  0.22366335],
       [ 0.40915328,  0.13880084,  0.4917931 ,  0.46405917,  0.12999071,
        -0.26662682, -0.01618852, -0.32241027,  0.49195777, -0.05550008,
         0.25165016, -0.45587014, -0.3652724 , -0.30682303],
       [-0.24363765, -0.14795247, -0.22807015, -0.06573142, -0.07964268,
        -0.39137917, -0.43739123, -0.14493855, -0.4112111 , -0.06679969,
        -0.49143879,  0.03580102, -0.42651601,  0.19420591]])

## Predito

In [30]:
Xp = np.concatenate((x_test_norm, np.reshape(np.ones(len(x_test_norm)),(len(x_test_norm),1))), axis=1)

In [31]:
Y_pred =  np.zeros(y_test.shape)
for i in range(len(Xp[:,])):
    Y_pred[i] = f_predict(Xp[i],w_hidden_layer,w_output_layer)

In [32]:
np.argmax(Y_pred,axis=1)

array([1, 0, 1, 1, 1, 0, 2, 0, 1, 1, 1, 0, 1, 0, 0, 2, 1, 2, 2, 2, 1, 2,
       1, 1, 0, 2, 2, 1, 1, 0, 1, 1, 0, 1, 2, 1, 2, 2, 0, 1, 2, 1, 1, 0,
       2, 1, 1, 2, 2, 0, 2, 0, 1, 1])

## Real

In [33]:
np.argmax(np.array(y_test),axis=1)

array([1, 0, 1, 1, 1, 1, 2, 0, 1, 1, 2, 0, 1, 1, 1, 2, 1, 2, 2, 2, 1, 2,
       1, 1, 0, 2, 2, 1, 1, 0, 1, 1, 0, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 0,
       2, 1, 1, 2, 2, 0, 2, 0, 1, 1])

# MLP

## Matriz de Confusão

In [34]:
from sklearn.metrics import confusion_matrix
confusion_matrix(np.argmax(np.array(y_test),axis=1), np.argmax(Y_pred,axis=1))

array([[ 9,  0,  0],
       [ 4, 24,  1],
       [ 0,  1, 15]])

## Acurácia

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(np.argmax(np.array(y_test),axis=1), np.argmax(Y_pred,axis=1))

0.8888888888888888